# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.interpret import ExplanationClient
from azureml.pipeline.steps import AutoMLStep


In [ ]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

### Workspace 

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="automl-udacity-capstone")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

### Compute Cluster

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

ws = Workspace.from_config() # this automatically looks for a directory .azureml

#Create compute cluster
cluster_name="automl-project"

try:
  compute_target=ComputeTarget(workspace=ws,name=cluster_name)
  print("Found existing compute target")
except ComputeTargetException:
  print('Creating a new compute target')
  compute_config=AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                       max_nodes=5, 
                                                       idle_seconds_before_scaledown=2400,
                                                       vm_priority='lowpriority')
  compute_target=ComputeTarget.create(ws,cluster_name,compute_config)
  

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes =  2)

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# Import dataset

from azureml.core import Dataset

ds = Dataset.get_by_name(ws, name='Glass Dataset')
df = ds.to_pandas_dataframe()
df

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(df)
print(type(x))
print(type(y))
      ### YOUR DATA OBJECT HERE ###)

In [ ]:
import pandas as pd

df_y = pd.DataFrame(y,columns=['Type'])
df_y.head()

In [ ]:
final_df = pd.concat([x,df_y],axis=1)
final_df.head()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_hours" : 0.3,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 8,
    "max_cores_per_iteration": -1,
    "n_cross_validations": 5,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             training_data = final_df,
                             label_column_name = 'Type',
                             iterations= 100,
                             **automl_settings
                            )

In [ ]:
# TODO: Submit your experiment
automl_run = exp.submit(automl_config)

In [ ]:
automl_run

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

# Wait for the remote run to complete
automl_run.wait_for_completion()

In [ ]:
automl_run.get_metrics('accuracy')

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.


Retrieve the Best Model's explanation from the best_run which includes explanations for engineered features and raw features. Make sure that the run for generating explanations for the best model is completed.

In [ ]:
# Wait for the best model explanation run to complete
from azureml.core.run import Run
model_explainability_run_id = remote_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=experiment, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

# Get the best run object
automl_best_run, model = automl_run.get_output()
automl_best_run_metrics = automl_best_run.get_metrics()

In [ ]:
print(model)
print('Best Run Id: ', automl_best_run.id)
print('\n Accuracy:', automl_best_run_metrics['accuracy'])

#### Describing the Best model

In [ ]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(model)

In [ ]:
#TODO: Save the best model
import joblib
#best_model_automl=
joblib.dump(model,filename='outputs/best_automlmodel.pkl')

In [ ]:
automl_best_run.get_tags()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_path ='outputs/best_automlmodel.pkl',
                        model_name = "automl")

In [ ]:
model.id

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
model_name = best_run.properties['model_name']

env = bestrun.get_environment().save_to_directory(path='environments')

script_file_name = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1,
                                               enable_app_insights=True, 
                                               auth_enabled=True, 
                                               description = 'Service for Automl Classification')

aci_service_name = 'automl-capstone'
print(aci_service_name)

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

In [ ]:
aci_service.get_logs()

In [ ]:
print("scoring URI: " + aci_service.scoring_uri)
print("Swagger URI: " + aci_service.swagger_uri)
print("Authetication Key: " + aci_service.get_keys()[0])

#### Call the services

In [ ]:
import requests
import json

# URL for the web service
scoring_uri = aci_service.swagger_uri #'<your web service URI>'
# If the service is authenticated, set the key or token
key =  aci_service.get_keys()[0]  #'<your key or token>'

# Two sets of data to score, so we get two results back
data = {"data":
        [
            {
               'RI': 5,
               'Na': 2,
               'Mg': 3.5,
               'Al': 0.7,
               'Si': 83,
               'K': 1.2,
               'Ca': 9,
               'Ba': 0.12,
               'Fe': 0.15
               
               },
            {
               'RI': 2,
               'Na': 25,
               'Mg': 2.5,
               'Al': 1.7,
               'Si': 63,
               'K': 2.2,
               'Ca': 6,
               'Ba': 0.15,
               'Fe': 0.18   
           }
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

### Testing the deployed service

In [ ]:
%run endpoint.py

In [ ]:
data_test = final_df[210:215]

In [ ]:
y_true = data_test.pop('Type')
sample_json = json.dumps({'data':data_test.to_dict(orient='records')})
print(sample_json)

In [ ]:
output = aci_service.run(sample_json)
print('Prediction: ', output)
print('True Values: ', y_true.values)

### Delete a Web Service

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
aci_service.delete()

#### Deleting compute

In [ ]:
compute_target.delete()